# 3 Pointers Made against game_totals.csv

### Import packages

In [132]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
from scipy.stats import pearsonr
import itertools

pd.set_option("display.max_columns", None)

### Set working directory

In [133]:
# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

# Change working directory
os.chdir('/Users/tyler/OneDrive/Documents/Python/NBA')

# Print working directory
cwd = os.getcwd()
print(f'Directory: {cwd}')

Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA
Directory: C:\Users\tyler\OneDrive\Documents\Python\NBA


## Exploratory Data Analysis

### Import data

In [134]:
df = pd.read_csv('backend/data/totals/game_totals.csv').drop(['Unnamed: 0'], axis=1)

### Basic exploration

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36248 entries, 0 to 36247
Data columns (total 22 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     36248 non-null  object 
 1   visitor  36248 non-null  object 
 2   home     36248 non-null  object 
 3   team     36248 non-null  int64  
 4   fg       36248 non-null  int64  
 5   fga      36248 non-null  int64  
 6   fg_perc  36248 non-null  float64
 7   3p       36248 non-null  int64  
 8   3pa      36248 non-null  int64  
 9   3p_perc  36248 non-null  float64
 10  ft       36248 non-null  int64  
 11  fta      36248 non-null  int64  
 12  ft_perc  36248 non-null  float64
 13  orb      36248 non-null  int64  
 14  drb      36248 non-null  int64  
 15  trb      36248 non-null  int64  
 16  ast      36248 non-null  int64  
 17  stl      36248 non-null  int64  
 18  blk      36248 non-null  int64  
 19  tov      36248 non-null  int64  
 20  pf       36248 non-null  int64  
 21  pts      362

In [136]:
df.sample(5)

,date,visitor,home,team,fg,fga,fg_perc,3p,3pa,3p_perc,ft,fta,ft_perc,orb,drb,trb,ast,stl,blk,tov,pf,pts
17458,"Sat, Mar 30, 2013",Los Angeles Lakers,Sacramento Kings,0,38,82,0.463,10,30,0.333,17,26,0.654,15,27,42,28,9,7,7,16,103
5100,"Tue, Apr 22, 2008",Toronto Raptors,Orlando Magic,0,37,81,0.457,11,29,0.379,18,20,0.900,7,33,40,21,4,1,13,23,103
27203,"Sat, Jan 28, 2017",Boston Celtics,Milwaukee Bucks,1,37,87,0.425,9,21,0.429,25,32,0.781,9,38,47,23,6,6,15,25,108
32180,"Fri, Jan 4, 2019",Los Angeles Clippers,Phoenix Suns,0,43,84,0.512,10,24,0.417,25,32,0.781,16,27,43,31,12,7,20,27,121
3836,"Wed, Jan 23, 2008",Dallas Mavericks,Charlotte Bobcats,0,39,82,0.476,3,17,0.176,21,25,0.840,10,33,43,18,4,6,6,20,102


In [137]:
# Convert 'date' column to Date object
df['date'] = pd.to_datetime(df['date'])

# Conver 'team' column to Team Name
df['team'] = np.where(df['team'], df['home'], df['visitor'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36248 entries, 0 to 36247
Data columns (total 22 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     36248 non-null  datetime64[ns]
 1   visitor  36248 non-null  object        
 2   home     36248 non-null  object        
 3   team     36248 non-null  object        
 4   fg       36248 non-null  int64         
 5   fga      36248 non-null  int64         
 6   fg_perc  36248 non-null  float64       
 7   3p       36248 non-null  int64         
 8   3pa      36248 non-null  int64         
 9   3p_perc  36248 non-null  float64       
 10  ft       36248 non-null  int64         
 11  fta      36248 non-null  int64         
 12  ft_perc  36248 non-null  float64       
 13  orb      36248 non-null  int64         
 14  drb      36248 non-null  int64         
 15  trb      36248 non-null  int64         
 16  ast      36248 non-null  int64         
 17  stl      36248 non-null  int64 

# Dataframe of team's last 15 performances

In [138]:
# Return ten lastest dates team played
def last_15_date(team, date):
    schedule = df[df['team'] == team].sort_values(by='date').reset_index()
    date_index = schedule[schedule['date'] == date].index[0]
    if date_index - 15 < 0:
        return None, None, None, None, None, None, None, None, None, None, None, None, None, None, None
    else:
        date_1, date_2 = schedule.iloc[date_index - 1]['date'], schedule.iloc[date_index - 2]['date']
        date_3, date_4 = schedule.iloc[date_index - 3]['date'], schedule.iloc[date_index - 4]['date']
        date_5, date_6 = schedule.iloc[date_index - 5]['date'], schedule.iloc[date_index - 6]['date']
        date_7, date_8 = schedule.iloc[date_index - 7]['date'], schedule.iloc[date_index - 8]['date']
        date_9, date_10 = schedule.iloc[date_index - 9]['date'], schedule.iloc[date_index - 10]['date']
        date_11, date_12 = schedule.iloc[date_index - 11]['date'], schedule.iloc[date_index - 12]['date']
        date_13, date_14 = schedule.iloc[date_index - 13]['date'], schedule.iloc[date_index - 14]['date']
        date_15 = schedule.iloc[date_index - 15]['date']
        return date_1, date_2, date_3, date_4, date_5, date_6, date_7, date_8, date_9, date_10, date_11, date_12, date_13, date_14, date_15

df['dates'] = df.apply(lambda x: last_15_date(x.team, x.date), axis=1)
df['date_1'], df['date_2'] = df['dates'].apply(lambda x: x[0]), df['dates'].apply(lambda x: x[1])
df['date_3'], df['date_4'] = df['dates'].apply(lambda x: x[2]), df['dates'].apply(lambda x: x[3])
df['date_5'], df['date_6'] = df['dates'].apply(lambda x: x[4]), df['dates'].apply(lambda x: x[5])
df['date_7'], df['date_8'] = df['dates'].apply(lambda x: x[6]), df['dates'].apply(lambda x: x[7])
df['date_9'], df['date_10'] = df['dates'].apply(lambda x: x[8]), df['dates'].apply(lambda x: x[9])
df['date_11'], df['date_12'] = df['dates'].apply(lambda x: x[10]), df['dates'].apply(lambda x: x[11])
df['date_13'], df['date_14'] = df['dates'].apply(lambda x: x[12]), df['dates'].apply(lambda x: x[13])
df['date_15'] = df['dates'].apply(lambda x: x[14])

In [139]:
# X and y column names to merge on
y_cols = df.columns
x_cols = ['date', 'team', 'fg', 'fga', 'fg_perc', '3p', '3pa', '3p_perc', 'ft', 'fta', 'ft_perc', 
          'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']

last_15_games = df[y_cols]
last_15_games['target'] = last_15_games['3p']
X = df[x_cols]

# Dataframe of target (3pt made by each team) and of variables (last 5 games stats for each team)
weeks = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
for week in weeks:
    last_15_games = pd.merge(last_15_games, X, left_on=['date' + week, 'team'], right_on=['date', 'team'], how='left', suffixes=('', week))

## Feature Engineering

In [140]:
# Create season variable
def season(month, year):
    if (month in list(range(10, 13)) and year == 2006) or (month in list(range(1, 7)) and year == 2007):
        return 2006
    elif (month in list(range(10, 13)) and year == 2007) or (month in list(range(1, 7)) and year == 2008):
        return 2007
    elif (month in list(range(10, 13)) and year == 2008) or (month in list(range(1, 7)) and year == 2009):
        return 2008
    elif (month in list(range(10, 13)) and year == 2009) or (month in list(range(1, 7)) and year == 2010):
        return 2009
    elif (month in list(range(10, 13)) and year == 2010) or (month in list(range(1, 7)) and year == 2011):
        return 2010
    elif (month in list(range(10, 11)) and year == 2011) or (month in list(range(1, 7)) and year == 2012):
        return 2011
    elif (month in list(range(10, 13)) and year == 2012) or (month in list(range(1, 7)) and year == 2013):
        return 2012
    elif (month in list(range(10, 13)) and year == 2013) or (month in list(range(1, 7)) and year == 2014):
        return 2013
    elif (month in list(range(10, 13)) and year == 2014) or (month in list(range(1, 7)) and year == 2015):
        return 2014
    elif (month in list(range(10, 13)) and year == 2015) or (month in list(range(1, 7)) and year == 2016):
        return 2015
    elif (month in list(range(10, 13)) and year == 2016) or (month in list(range(1, 7)) and year == 2017):
        return 2016
    elif (month in list(range(10, 13)) and year == 2017) or (month in list(range(1, 7)) and year == 2018):
        return 2017
    elif (month in list(range(10, 13)) and year == 2018) or (month in list(range(1, 7)) and year == 2019):
        return 2018
    elif (month in list(range(10, 13)) and year == 2019) or (month in list(range(1, 11)) and year == 2020):
        return 2019
    elif (month in list(range(12, 13)) and year == 2020) or (month in list(range(1, 8)) and year == 2021):
        return 2020

df['season'] = df['date'].apply(lambda x: season(x.month, x.year))
last_15_games['season'] = last_15_games['date'].apply(lambda x: season(x.month, x.year))
last_15_games.sample()

,date,visitor,home,team,fg,fga,fg_perc,3p,3pa,3p_perc,ft,fta,ft_perc,orb,drb,trb,ast,stl,blk,tov,pf,pts,dates,date_1,date_2,date_3,date_4,date_5,date_6,date_7,date_8,date_9,date_10,date_11,date_12,date_13,date_14,date_15,target,date_1,fg_1,fga_1,fg_perc_1,3p_1,3pa_1,3p_perc_1,ft_1,fta_1,ft_perc_1,orb_1,drb_1,trb_1,ast_1,stl_1,blk_1,tov_1,pf_1,pts_1,date_2,fg_2,fga_2,fg_perc_2,3p_2,3pa_2,3p_perc_2,ft_2,fta_2,ft_perc_2,orb_2,drb_2,trb_2,ast_2,stl_2,blk_2,tov_2,pf_2,pts_2,date_3,fg_3,fga_3,fg_perc_3,3p_3,3pa_3,3p_perc_3,ft_3,fta_3,ft_perc_3,orb_3,drb_3,trb_3,ast_3,stl_3,blk_3,tov_3,pf_3,pts_3,date_4,fg_4,fga_4,fg_perc_4,3p_4,3pa_4,3p_perc_4,ft_4,fta_4,ft_perc_4,orb_4,drb_4,trb_4,ast_4,stl_4,blk_4,tov_4,pf_4,pts_4,date_5,fg_5,fga_5,fg_perc_5,3p_5,3pa_5,3p_perc_5,ft_5,fta_5,ft_perc_5,orb_5,drb_5,trb_5,ast_5,stl_5,blk_5,tov_5,pf_5,pts_5,date_6,fg_6,fga_6,fg_perc_6,3p_6,3pa_6,3p_perc_6,ft_6,fta_6,ft_perc_6,orb_6,drb_6,trb_6,ast_6,stl_6,blk_6,tov_6,pf_6,pts_6,date_7,fg_7,fga_7,fg_perc_7,3p_7,3pa_7,3p_perc_7,ft_7,fta_7,ft_perc_7,orb_7,drb_7,trb_7,ast_7,stl_7,blk_7,tov_7,pf_7,pts_7,date_8,fg_8,fga_8,fg_perc_8,3p_8,3pa_8,3p_perc_8,ft_8,fta_8,ft_perc_8,orb_8,drb_8,trb_8,ast_8,stl_8,blk_8,tov_8,pf_8,pts_8,date_9,fg_9,fga_9,fg_perc_9,3p_9,3pa_9,3p_perc_9,ft_9,fta_9,ft_perc_9,orb_9,drb_9,trb_9,ast_9,stl_9,blk_9,tov_9,pf_9,pts_9,date_10,fg_10,fga_10,fg_perc_10,3p_10,3pa_10,3p_perc_10,ft_10,fta_10,ft_perc_10,orb_10,drb_10,trb_10,ast_10,stl_10,blk_10,tov_10,pf_10,pts_10,date_11,fg_11,fga_11,fg_perc_11,3p_11,3pa_11,3p_perc_11,ft_11,fta_11,ft_perc_11,orb_11,drb_11,trb_11,ast_11,stl_11,blk_11,tov_11,pf_11,pts_11,date_12,fg_12,fga_12,fg_perc_12,3p_12,3pa_12,3p_perc_12,ft_12,fta_12,ft_perc_12,orb_12,drb_12,trb_12,ast_12,stl_12,blk_12,tov_12,pf_12,pts_12,date_13,fg_13,fga_13,fg_perc_13,3p_13,3pa_13,3p_perc_13,ft_13,fta_13,ft_perc_13,orb_13,drb_13,trb_13,ast_13,stl_13,blk_13,tov_13,pf_13,pts_13,date_14,fg_14,fga_14,fg_perc_14,3p_14,3pa_14,3p_perc_14,ft_14,fta_14,ft_perc_14,orb_14,drb_14,trb_14,ast_14,stl_14,blk_14,tov_14,pf_14,pts_14,date_15,fg_15,fga_15,fg_perc_15,3p_15,3pa_15,3p_perc_15,ft_15,fta_15,ft_perc_15,orb_15,drb_15,trb_15,ast_15,stl_15,blk_15,tov_15,pf_15,pts_15,season
24260,2016-01-09,Charlotte Hornets,Los Angeles Clippers,Charlotte Hornets,31,84,0.369,7,23,0.304,14,19,0.737,11,36,47,14,4,3,15,16,83,"(2016-01-06 00:00:00, 2016-01-04 00:00:00, 201...",2016-01-06,2016-01-04,2016-01-02,2016-01-01,2015-12-30,2015-12-28,2015-12-26,2015-12-23,2015-12-21,2015-12-19,2015-12-17,2015-12-16,2015-12-12,2015-12-11,2015-12-09,7,2016-01-06,38.0,87.0,0.437,11.0,28.0,0.393,15.0,22.0,0.682,9.0,35.0,44.0,20.0,9.0,5.0,13.0,15.0,102.0,2016-01-04,38.0,86.0,0.442,7.0,21.0,0.333,18.0,19.0,0.947,8.0,33.0,41.0,15.0,7.0,1.0,11.0,12.0,101.0,2016-01-02,29.0,84.0,0.345,6.0,22.0,0.273,26.0,34.0,0.765,7.0,27.0,34.0,18.0,7.0,7.0,4.0,27.0,90.0,2016-01-01,34.0,85.0,0.4,10.0,32.0,0.313,16.0,18.0,0.889,14.0,29.0,43.0,15.0,7.0,7.0,10.0,16.0,94.0,2015-12-30,38.0,88.0,0.432,9.0,30.0,0.3,32.0,36.0,0.889,17.0,25.0,42.0,20.0,8.0,3.0,11.0,26.0,117.0,2015-12-28,43.0,89.0,0.483,6.0,25.0,0.24,16.0,22.0,0.727,7.0,39.0,46.0,28.0,7.0,6.0,10.0,16.0,108.0,2015-12-26,28.0,69.0,0.406,14.0,32.0,0.438,28.0,32.0,0.875,7.0,36.0,43.0,21.0,10.0,13.0,16.0,24.0,98.0,2015-12-23,29.0,81.0,0.358,8.0,28.0,0.286,23.0,32.0,0.719,12.0,32.0,44.0,19.0,6.0,6.0,16.0,18.0,89.0,2015-12-21,29.0,83.0,0.349,12.0,41.0,0.293,25.0,31.0,0.806,9.0,26.0,35.0,20.0,8.0,3.0,11.0,27.0,95.0,2015-12-19,39.0,93.0,0.419,10.0,33.0,0.303,13.0,16.0,0.813,18.0,33.0,51.0,23.0,8.0,4.0,17.0,20.0,101.0,2015-12-17,37.0,85.0,0.435,10.0,29.0,0.345,25.0,31.0,0.806,6.0,45.0,51.0,23.0,5.0,9.0,13.0,23.0,109.0,2015-12-16,36.0,79.0,0.456,14.0,34.0,0.412,12.0,15.0,0.8,7.0,26.0,33.0,23.0,12.0,3.0,16.0,19.0,98.0,2015-12-12,36.0,88.0,0.409,11.0,35.0,0.314,10.0,10.0,1.0,7.0,45.0,52.0,17.0,1.0,5.0,11.0,12.0,93.0,2015-12-11,39.0,78.0,0.5,18.0,37.0,0.486,27.0,30.0,0.9,7.0,34.0,41.0,23.0,5.0,5.0,13.0,24.0,123.0,2015-12-09,40.0,81.0,0.494,11.0,27.0,0.407,8

In [141]:
# Calculate z-score
def z_score(value, mean, std):
    return (value - mean) / std

In [142]:
# Season average for each stat
season_avgs = df.groupby(['team', 'season']).aggregate(['mean', 'std']).reset_index()
season_avgs.columns = ["_".join([tup[0], tup[1]]) if len(tup[1]) > 0 else tup[0] for tup in season_avgs.columns]
last_15_games = pd.merge(last_15_games, season_avgs, left_on=['team', 'season'], right_on=['team', 'season'])

In [143]:
stats = ['fg', 'fga', 'fg_perc', '3p', '3pa', '3p_perc', 'ft', 'fta', 'ft_perc', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']

for stat in stats:
    # 3 day trend
    last_15_games[stat + '_trend_3'] = \
        (z_score(last_15_games[stat + '_1'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_2'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_3'], last_15_games[stat + '_mean'], last_15_games[stat + '_std'])) / 3
    
    # 5 day trend
    last_15_games[stat + '_trend_5'] = \
        (z_score(last_15_games[stat + '_1'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_2'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_3'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_4'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_5'], last_15_games[stat + '_mean'], last_15_games[stat + '_std'])) / 5
    
    # 10 day trend
    last_15_games[stat + '_trend_10'] = \
        (z_score(last_15_games[stat + '_1'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_2'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_3'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_4'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_5'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_6'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_7'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_8'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_9'], last_15_games[stat + '_mean'], last_15_games[stat + '_std']) + \
         z_score(last_15_games[stat + '_10'], last_15_games[stat + '_mean'], last_15_games[stat + '_std'])) / 10

### Last Performance

In [144]:
stats = ['fg', 'fga', 'fg_perc', '3p', '3pa', '3p_perc', 'ft', 'fta', 'ft_perc', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']
dates = ['_1']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_game = last_15_games[cols].copy()

for stat in stats:
    last_game[stat] = 0
    for date in dates:
        last_game[stat] = last_game[stat] + last_game[stat + date]
    
    last_game[stat] = last_game[stat] / len(dates)

last_game = last_game.groupby(['date', 'visitor', 'home']).aggregate(['mean', 'sum'])

last_game_cols = [col 
                  for col in last_game.columns
                  if ('_perc' in col[0] and 'mean' == col[1]) or ('_perc' not in col[0] and 'sum' == col[1])]

last_game = last_game[last_game_cols].dropna(axis=0)
last_game.columns = [col[0] for col in last_game.columns]
last_game.head()

target  fg_1  fga_1  \
date       visitor         home                                          
2006-11-28 Indiana Pacers  Portland Trail Blazers      13  40.0   79.0   
           New York Knicks Chicago Bulls                9  34.0   73.0   
2006-11-29 Indiana Pacers  Golden State Warriors       18  79.0  156.0   
           New York Knicks Cleveland Cavaliers         13  33.0   69.0   
           Orlando Magic   Seattle SuperSonics          8  63.0  162.0   

                                                   fg_perc_1  3p_1  3pa_1  \
date       visitor         home                                             
2006-11-28 Indiana Pacers  Portland Trail Blazers     0.5060   8.0   12.0   
           New York Knicks Chicago Bulls              0.4660   3.0    9.0   
2006-11-29 Indiana Pacers  Golden State Warriors      0.5075  18.0   41.0   
           New York Knicks Cleveland Cavaliers        0.4780   2.0    9.0   
           Orlando Magic   Seattle SuperSonics        0.3895   3.0   21.0   

                                                   3p_perc_1  ft_1  fta_1  \
date       visitor         home                                             
2006-11-28 Indiana Pacers  Portland Trail Blazers      0.667  13.0   19.0   
           New York Knicks Chicago Bulls               0.333  24.0   38.0   
2006-11-29 Indiana Pacers  Golden State Warriors       0.452  40.0   48.0   
           New York Knicks Cleveland Cavaliers         0.222  17.0   30.0   
           Orlando Magic   Seattle SuperSonics         0.141  37.0   50.0   

                                                   ft_perc_1  orb_1  drb_1  \
date       visitor         home                                              
2006-11-28 Indiana Pacers  Portland Trail Blazers     0.6840    8.0   27.0   
           New York Knicks Chicago Bulls              0.6320   15.0   34.0   
2006-11-29 Indiana Pacers  Golden State Warriors      0.8330   16.0   54.0   
           New York Knicks Cleveland Cavaliers        0.5670    7.0   29.0   
           Orlando Magic   Seattle SuperSonics        0.7265   31.0   48.0   

                                                   trb_1  ast_1  stl_1  blk_1  \
date       visitor         home                                                 
2006-11-28 Indiana Pacers  Portland Trail Blazers   35.0   22.0    6.0    4.0   
           New York Knicks Chicago Bulls            49.0   19.0    4.0    4.0   
2006-11-29 Indiana Pacers  Golden State Warriors    70.0   55.0   14.0   19.0   
           New York Knicks Cleveland Cavaliers      36.0   19.0   11.0    6.0   
           Orlando Magic   Seattle SuperSonics      79.0   28.0   21.0    7.0   

                                                   tov_1  pf_1  pts_1  \
date       visitor         home                                         
2006-11-28 Indiana Pacers  Portland Trail Blazers   15.0  21.0  101.0   
           New York Knicks Chicago Bulls            25.0  32.0   95.0   
2006-11-29 Indiana Pacers  Golden State Warriors    25.0  41.0  216.0   
           New York Knicks Cleveland Cavaliers      22.0  27.0   85.0   
           Orlando Magic   Seattle SuperSonics      24.0  34.0  166.0   

                                                   fg_trend_3  fg_trend_5  \
date       visitor         home                                             
2006-11-28 Indiana Pacers  Portland Trail Blazers    0.281406    0.120755   
           New York Knicks Chicago Bulls            -0.461002   -0.490709   
2006-11-29 Indiana Pacers  Golden State Warriors     0.200973    0.354571   
           New York Knicks Cleveland Cavaliers      -0.386734   -0.535269   
           Orlando Magic   Seattle SuperSonics      -0.453125   -0.568333   

                                                   fg_trend_10  fga_trend_3  \
date       visitor         home                                               
2006-11-28 Indiana Pacers  Portland Trail Blazers    -0.339290    -0.293914   
           New York Knicks Chicago Bulls        

### Last 5 Performances

In [145]:
stats = ['fg', 'fga', 'fg_perc', '3p', '3pa', '3p_perc', 'ft', 'fta', 'ft_perc', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']
dates = ['_1', '_2', '_3', '_4', '_5']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_5_games = last_15_games[cols].copy()

for stat in stats:
    last_5_games[stat] = 0
    for date in dates:
        last_5_games[stat] = last_5_games[stat] + last_5_games[stat + date]
    
    last_5_games[stat] = last_5_games[stat] / len(dates)

last_5_games = last_5_games.groupby(['date', 'visitor', 'home']).aggregate(['mean', 'sum'])

last_5_game_cols = [col 
                    for col in last_5_games.columns
                    if ('_perc' in col[0] and 'mean' == col[1]) or ('_perc' not in col[0] and 'sum' == col[1])]

last_5_games = last_5_games[last_5_game_cols].dropna(axis=0)
last_5_games.columns = [col[0] for col in last_5_games.columns]
last_5_games.head()

target  fg_1  fg_2  fg_3  \
date       visitor         home                                               
2006-11-28 Indiana Pacers  Portland Trail Blazers      13  40.0  32.0  37.0   
           New York Knicks Chicago Bulls                9  34.0  34.0  32.0   
2006-11-29 Indiana Pacers  Golden State Warriors       18  79.0  65.0  83.0   
           New York Knicks Cleveland Cavaliers         13  33.0  34.0  34.0   
           Orlando Magic   Seattle SuperSonics          8  63.0  73.0  74.0   

                                                   fg_4  fg_5  fga_1  fga_2  \
date       visitor         home                                               
2006-11-28 Indiana Pacers  Portland Trail Blazers  38.0  31.0   79.0   70.0   
           New York Knicks Chicago Bulls           32.0  34.0   73.0   68.0   
2006-11-29 Indiana Pacers  Golden State Warriors   73.0  82.0  156.0  167.0   
           New York Knicks Cleveland Cavaliers     32.0  32.0   69.0   73.0   
           Orlando Magic   Seattle SuperSonics     66.0  71.0  162.0  159.0   

                                                   fga_3  fga_4  fga_5  \
date       visitor         home                                          
2006-11-28 Indiana Pacers  Portland Trail Blazers   78.0   76.0   66.0   
           New York Knicks Chicago Bulls            73.0   86.0   86.0   
2006-11-29 Indiana Pacers  Golden State Warriors   169.0  171.0  170.0   
           New York Knicks Cleveland Cavaliers      68.0   73.0   86.0   
           Orlando Magic   Seattle SuperSonics     165.0  144.0  161.0   

                                                   fg_perc_1  fg_perc_2  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers     0.5060      0.457   
           New York Knicks Chicago Bulls              0.4660      0.500   
2006-11-29 Indiana Pacers  Golden State Warriors      0.5075      0.388   
           New York Knicks Cleveland Cavaliers        0.4780      0.466   
           Orlando Magic   Seattle SuperSonics        0.3895      0.459   

                                                   fg_perc_3  fg_perc_4  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers     0.4740      0.500   
           New York Knicks Chicago Bulls              0.4380      0.372   
2006-11-29 Indiana Pacers  Golden State Warriors      0.4825      0.425   
           New York Knicks Cleveland Cavaliers        0.5000      0.438   
           Orlando Magic   Seattle SuperSonics        0.4485      0.460   

                                                   fg_perc_5  3p_1  3p_2  \
date       visitor         home                                            
2006-11-28 Indiana Pacers  Portland Trail Blazers     0.4700   8.0   5.0   
           New York Knicks Chicago Bulls              0.3950   3.0   4.0   
2006-11-29 Indiana Pacers  Golden State Warriors      0.4795  18.0  13.0   
           New York Knicks Cleveland Cavaliers        0.3720   2.0   3.0   
           Orlando Magic   Seattle SuperSonics        0.4410   3.0  13.0   

                                                   3p_3  3p_4  3p_5  3pa_1  \
date       visitor         home                                              
2006-11-28 Indiana Pacers  Portland Trail Blazers   5.0   4.0   4.0   12.0   
           New York Knicks Chicago Bulls            2.0   4.0   8.0    9.0   
2006-11-29 Indiana Pacers  Golden State Warriors   12.0  15.0  15.0   41.0   
           New York Knicks Cleveland Cavaliers      4.0   2.0   4.0    9.0   
           Orlando Magic   Seattle SuperSonics      8.0  12.0  10.0   21.0   

                                                   3pa_2  3pa_3  3pa_4  3pa_5  \
date       visitor         home                                                 
2006-11-28 Indiana Pacers  Portland Trail Blazers   20.0   15.0   14.0   14.0   
           New York Knicks Chicago Bulls         

### Last 10 Perfomances

In [146]:
stats = ['fg', 'fga', 'fg_perc', '3p', '3pa', '3p_perc', 'ft', 'fta', 'ft_perc', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_10_games = last_15_games[cols].copy()

for stat in stats:
    last_10_games[stat] = 0
    for date in dates:
        last_10_games[stat] = last_10_games[stat] + last_10_games[stat + date]
    
    last_10_games[stat] = last_10_games[stat] / len(dates)

last_10_games = last_10_games.groupby(['date', 'visitor', 'home']).aggregate(['mean', 'sum'])

last_10_game_cols = [col 
                    for col in last_10_games.columns
                    if ('_perc' in col[0] and 'mean' == col[1]) or ('_perc' not in col[0] and 'sum' == col[1])]

last_10_games = last_10_games[last_10_game_cols].dropna(axis=0)
last_10_games.columns = [col[0] for col in last_10_games.columns]
last_10_games.head()

target  fg_1  fg_2  fg_3  \
date       visitor         home                                               
2006-11-28 Indiana Pacers  Portland Trail Blazers      13  40.0  32.0  37.0   
           New York Knicks Chicago Bulls                9  34.0  34.0  32.0   
2006-11-29 Indiana Pacers  Golden State Warriors       18  79.0  65.0  83.0   
           New York Knicks Cleveland Cavaliers         13  33.0  34.0  34.0   
           Orlando Magic   Seattle SuperSonics          8  63.0  73.0  74.0   

                                                   fg_4  fg_5  fg_6  fg_7  \
date       visitor         home                                             
2006-11-28 Indiana Pacers  Portland Trail Blazers  38.0  31.0  33.0  30.0   
           New York Knicks Chicago Bulls           32.0  34.0  40.0  37.0   
2006-11-29 Indiana Pacers  Golden State Warriors   73.0  82.0  77.0  83.0   
           New York Knicks Cleveland Cavaliers     32.0  32.0  34.0  40.0   
           Orlando Magic   Seattle SuperSonics     66.0  71.0  73.0  69.0   

                                                   fg_8  fg_9  fg_10  fga_1  \
date       visitor         home                                               
2006-11-28 Indiana Pacers  Portland Trail Blazers  31.0  35.0   28.0   79.0   
           New York Knicks Chicago Bulls           39.0  36.0   35.0   73.0   
2006-11-29 Indiana Pacers  Golden State Warriors   73.0  72.0   70.0  156.0   
           New York Knicks Cleveland Cavaliers     37.0  39.0   36.0   69.0   
           Orlando Magic   Seattle SuperSonics     82.0  85.0   68.0  162.0   

                                                   fga_2  fga_3  fga_4  fga_5  \
date       visitor         home                                                 
2006-11-28 Indiana Pacers  Portland Trail Blazers   70.0   78.0   76.0   66.0   
           New York Knicks Chicago Bulls            68.0   73.0   86.0   86.0   
2006-11-29 Indiana Pacers  Golden State Warriors   167.0  169.0  171.0  170.0   
           New York Knicks Cleveland Cavaliers      73.0   68.0   73.0   86.0   
           Orlando Magic   Seattle SuperSonics     159.0  165.0  144.0  161.0   

                                                   fga_6  fga_7  fga_8  fga_9  \
date       visitor         home                                                 
2006-11-28 Indiana Pacers  Portland Trail Blazers   77.0   74.0   73.0   76.0   
           New York Knicks Chicago Bulls            77.0   83.0   80.0   79.0   
2006-11-29 Indiana Pacers  Golden State Warriors   166.0  165.0  178.0  141.0   
           New York Knicks Cleveland Cavaliers      86.0   77.0   83.0   80.0   
           Orlando Magic   Seattle SuperSonics     147.0  152.0  162.0  176.0   

                                                   fga_10  fg_perc_1  \
date       visitor         home                                        
2006-11-28 Indiana Pacers  Portland Trail Blazers    67.0     0.5060   
           New York Knicks Chicago Bulls             81.0     0.4660   
2006-11-29 Indiana Pacers  Golden State Warriors    155.0     0.5075   
           New York Knicks Cleveland Cavaliers       79.0     0.4780   
           Orlando Magic   Seattle SuperSonics      149.0     0.3895   

                                                   fg_perc_2  fg_perc_3  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers      0.457     0.4740   
           New York Knicks Chicago Bulls               0.500     0.4380   
2006-11-29 Indiana Pacers  Golden State Warriors       0.388     0.4825   
           New York Knicks Cleveland Cavaliers         0.466     0.5000   
           Orlando Magic   Seattle SuperSonics         0.459     0.4485   

                                                   fg_perc_4  fg_perc_5  \
date       visitor         home                                           
2006-11-28 Indiana Pacers  Portland Trail Blazers      0.500     0.4700   
           

### Last 15 Performances (Unweighted)

In [147]:
stats = ['fg', 'fga', 'fg_perc', '3p', '3pa', '3p_perc', 'ft', 'fta', 'ft_perc', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_15_games_unweighted = last_15_games[cols].copy()

for stat in stats:
    last_15_games_unweighted[stat] = 0
    for date in dates:
        last_15_games_unweighted[stat] = last_15_games_unweighted[stat] + last_15_games_unweighted[stat + date]
    
    last_15_games_unweighted[stat] = last_15_games_unweighted[stat] / len(dates)

last_15_games_unweighted = last_15_games_unweighted.groupby(['date', 'visitor', 'home']).aggregate(['mean', 'sum'])

last_15_game_cols = [col 
                     for col in last_15_games_unweighted.columns
                     if ('_perc' in col[0] and 'mean' == col[1]) or ('_perc' not in col[0] and 'sum' == col[1])]

last_15_games_unweighted = last_15_games_unweighted[last_15_game_cols].dropna(axis=0)
last_15_games_unweighted.columns = [col[0] for col in last_15_games_unweighted.columns]
last_15_games_unweighted.head()

target  fg_1  fg_2  fg_3  \
date       visitor         home                                               
2006-11-28 Indiana Pacers  Portland Trail Blazers      13  40.0  32.0  37.0   
           New York Knicks Chicago Bulls                9  34.0  34.0  32.0   
2006-11-29 Indiana Pacers  Golden State Warriors       18  79.0  65.0  83.0   
           New York Knicks Cleveland Cavaliers         13  33.0  34.0  34.0   
           Orlando Magic   Seattle SuperSonics          8  63.0  73.0  74.0   

                                                   fg_4  fg_5  fg_6  fg_7  \
date       visitor         home                                             
2006-11-28 Indiana Pacers  Portland Trail Blazers  38.0  31.0  33.0  30.0   
           New York Knicks Chicago Bulls           32.0  34.0  40.0  37.0   
2006-11-29 Indiana Pacers  Golden State Warriors   73.0  82.0  77.0  83.0   
           New York Knicks Cleveland Cavaliers     32.0  32.0  34.0  40.0   
           Orlando Magic   Seattle SuperSonics     66.0  71.0  73.0  69.0   

                                                   fg_8  fg_9  fg_10  fg_11  \
date       visitor         home                                               
2006-11-28 Indiana Pacers  Portland Trail Blazers  31.0  35.0   28.0   35.0   
           New York Knicks Chicago Bulls           39.0  36.0   35.0   40.0   
2006-11-29 Indiana Pacers  Golden State Warriors   73.0  72.0   70.0   69.0   
           New York Knicks Cleveland Cavaliers     37.0  39.0   36.0   35.0   
           Orlando Magic   Seattle SuperSonics     82.0  85.0   68.0   66.0   

                                                   fg_12  fg_13  fg_14  fg_15  \
date       visitor         home                                                 
2006-11-28 Indiana Pacers  Portland Trail Blazers   32.0   35.0   32.0   40.0   
           New York Knicks Chicago Bulls            32.0   34.0   33.0   40.0   
2006-11-29 Indiana Pacers  Golden State Warriors    73.0   74.0   70.0   69.0   
           New York Knicks Cleveland Cavaliers      40.0   32.0   34.0   33.0   
           Orlando Magic   Seattle SuperSonics      70.0   72.0   83.0   76.0   

                                                   fga_1  fga_2  fga_3  fga_4  \
date       visitor         home                                                 
2006-11-28 Indiana Pacers  Portland Trail Blazers   79.0   70.0   78.0   76.0   
           New York Knicks Chicago Bulls            73.0   68.0   73.0   86.0   
2006-11-29 Indiana Pacers  Golden State Warriors   156.0  167.0  169.0  171.0   
           New York Knicks Cleveland Cavaliers      69.0   73.0   68.0   73.0   
           Orlando Magic   Seattle SuperSonics     162.0  159.0  165.0  144.0   

                                                   fga_5  fga_6  fga_7  fga_8  \
date       visitor         home                                                 
2006-11-28 Indiana Pacers  Portland Trail Blazers   66.0   77.0   74.0   73.0   
           New York Knicks Chicago Bulls            86.0   77.0   83.0   80.0   
2006-11-29 Indiana Pacers  Golden State Warriors   170.0  166.0  165.0  178.0   
           New York Knicks Cleveland Cavaliers      86.0   86.0   77.0   83.0   
           Orlando Magic   Seattle SuperSonics     161.0  147.0  152.0  162.0   

                                                   fga_9  fga_10  fga_11  \
date       visitor         home                                            
2006-11-28 Indiana Pacers  Portland Trail Blazers   76.0    67.0    68.0   
           New York Knicks Chicago Bulls            79.0    81.0    80.0   
2006-11-29 Indiana Pacers  Golden State Warriors   141.0   155.0   163.0   
           New York Knicks Cleveland Cavaliers      80.0    79.0    81.0   
           Orlando Magic   Seattle SuperSonics     176.0   149.0   140.0   

                                                   fga_12  fga_13  fga_14  \
date       visitor         home                                             
2006-11-28 I

### Last 15 Performances (Weighted)

In [148]:
stats = ['fg', 'fga', 'fg_perc', '3p', '3pa', '3p_perc', 'ft', 'fta', 'ft_perc', 
         'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts']
dates = ['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12', '_13', '_14', '_15']
trends = ['_trend_3', '_trend_5', '_trend_10']
cols = ['date', 'visitor', 'home', 'team', 'target'] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, dates))] + \
    [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]

last_15_games_weighted = last_15_games[cols].copy()

for stat in stats:
    last_15_games_weighted[stat] = 0
    for date in dates:
        weight = ((int(date.strip('_')) - 1) // 5) + 1
        last_15_games_weighted[stat] = (last_15_games_weighted[stat] + last_15_games_weighted[stat + date]) * weight
    
    last_15_games_weighted[stat] = last_15_games_weighted[stat] / len(dates)

last_15_games_weighted = last_15_games_weighted.groupby(['date', 'visitor', 'home']).aggregate(['mean', 'sum'])

last_15_game_cols = [col 
                     for col in last_15_games_weighted.columns
                     if ('_perc' in col[0] and 'mean' == col[1]) or ('_perc' not in col[0] and 'sum' == col[1])]

last_15_games_weighted = last_15_games_weighted[last_15_game_cols].dropna(axis=0)
last_15_games_weighted.columns = [col[0] for col in last_15_games_weighted.columns]
last_15_games_weighted.head()

target  fg_1  fg_2  fg_3  \
date       visitor         home                                               
2006-11-28 Indiana Pacers  Portland Trail Blazers      13  40.0  32.0  37.0   
           New York Knicks Chicago Bulls                9  34.0  34.0  32.0   
2006-11-29 Indiana Pacers  Golden State Warriors       18  79.0  65.0  83.0   
           New York Knicks Cleveland Cavaliers         13  33.0  34.0  34.0   
           Orlando Magic   Seattle SuperSonics          8  63.0  73.0  74.0   

                                                   fg_4  fg_5  fg_6  fg_7  \
date       visitor         home                                             
2006-11-28 Indiana Pacers  Portland Trail Blazers  38.0  31.0  33.0  30.0   
           New York Knicks Chicago Bulls           32.0  34.0  40.0  37.0   
2006-11-29 Indiana Pacers  Golden State Warriors   73.0  82.0  77.0  83.0   
           New York Knicks Cleveland Cavaliers     32.0  32.0  34.0  40.0   
           Orlando Magic   Seattle SuperSonics     66.0  71.0  73.0  69.0   

                                                   fg_8  fg_9  fg_10  fg_11  \
date       visitor         home                                               
2006-11-28 Indiana Pacers  Portland Trail Blazers  31.0  35.0   28.0   35.0   
           New York Knicks Chicago Bulls           39.0  36.0   35.0   40.0   
2006-11-29 Indiana Pacers  Golden State Warriors   73.0  72.0   70.0   69.0   
           New York Knicks Cleveland Cavaliers     37.0  39.0   36.0   35.0   
           Orlando Magic   Seattle SuperSonics     82.0  85.0   68.0   66.0   

                                                   fg_12  fg_13  fg_14  fg_15  \
date       visitor         home                                                 
2006-11-28 Indiana Pacers  Portland Trail Blazers   32.0   35.0   32.0   40.0   
           New York Knicks Chicago Bulls            32.0   34.0   33.0   40.0   
2006-11-29 Indiana Pacers  Golden State Warriors    73.0   74.0   70.0   69.0   
           New York Knicks Cleveland Cavaliers      40.0   32.0   34.0   33.0   
           Orlando Magic   Seattle SuperSonics      70.0   72.0   83.0   76.0   

                                                   fga_1  fga_2  fga_3  fga_4  \
date       visitor         home                                                 
2006-11-28 Indiana Pacers  Portland Trail Blazers   79.0   70.0   78.0   76.0   
           New York Knicks Chicago Bulls            73.0   68.0   73.0   86.0   
2006-11-29 Indiana Pacers  Golden State Warriors   156.0  167.0  169.0  171.0   
           New York Knicks Cleveland Cavaliers      69.0   73.0   68.0   73.0   
           Orlando Magic   Seattle SuperSonics     162.0  159.0  165.0  144.0   

                                                   fga_5  fga_6  fga_7  fga_8  \
date       visitor         home                                                 
2006-11-28 Indiana Pacers  Portland Trail Blazers   66.0   77.0   74.0   73.0   
           New York Knicks Chicago Bulls            86.0   77.0   83.0   80.0   
2006-11-29 Indiana Pacers  Golden State Warriors   170.0  166.0  165.0  178.0   
           New York Knicks Cleveland Cavaliers      86.0   86.0   77.0   83.0   
           Orlando Magic   Seattle SuperSonics     161.0  147.0  152.0  162.0   

                                                   fga_9  fga_10  fga_11  \
date       visitor         home                                            
2006-11-28 Indiana Pacers  Portland Trail Blazers   76.0    67.0    68.0   
           New York Knicks Chicago Bulls            79.0    81.0    80.0   
2006-11-29 Indiana Pacers  Golden State Warriors   141.0   155.0   163.0   
           New York Knicks Cleveland Cavaliers      80.0    79.0    81.0   
           Orlando Magic   Seattle SuperSonics     176.0   149.0   140.0   

                                                   fga_12  fga_13  fga_14  \
date       visitor         home                                             
2006-11-28 I

## Correlations of performances (last 1, last 5, last 10)

In [149]:
corr_df = pd.DataFrame()

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in last_15_games_weighted:
    corr_p = pearsonr(last_15_games_weighted['target'], last_15_games_weighted[col])
    row = {'last': '15_weighted', 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)

# Correlations for last 15 game stats vs 3pt made (unweighted)
for col in last_15_games_unweighted:
    corr_p = pearsonr(last_15_games_unweighted['target'], last_15_games_unweighted[col])
    row = {'last': '15_unweighted', 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)

# Correlations for last 10 game stats vs 3pt made 
for col in last_10_games:
    corr_p = pearsonr(last_10_games['target'], last_10_games[col])
    row = {'last': 10, 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Correlations for last 10 game stats vs 3pt made 
for col in last_5_games:
    corr_p = pearsonr(last_5_games['target'], last_5_games[col])
    row = {'last': 5, 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Correlations for last game stats vs 3pt made 
for col in last_game:
    corr_p = pearsonr(last_game['target'], last_game[col])
    row = {'last': 1, 'stat': col, 'corr': round(corr_p[0], 2), 'p-value': round(corr_p[1], 2)}
    corr_df = corr_df.append(row, ignore_index=True)
    
# Print each correlation
for stat in stats + [tup[0] + tup[1] for tup in list(itertools.product(stats, trends))]:
    print(f'Stat: {stat}')
    print(corr_df[(corr_df['stat'] == stat) & (corr_df['p-value'] < .05)].set_index(['last']).drop(['stat'], axis=1))
    print('\n')

Stat: fg
               corr  p-value
last                        
15_weighted    0.41      0.0
15_unweighted  0.43      0.0
10             0.42      0.0
5              0.39      0.0
1              0.27      0.0


Stat: fga
               corr  p-value
last                        
15_weighted    0.42      0.0
15_unweighted  0.43      0.0
10             0.42      0.0
5              0.41      0.0
1              0.31      0.0


Stat: fg_perc
               corr  p-value
last                        
15_weighted    0.10      0.0
15_unweighted  0.12      0.0
10             0.11      0.0
5              0.09      0.0
1              0.05      0.0


Stat: 3p
               corr  p-value
last                        
15_weighted    0.70      0.0
15_unweighted  0.71      0.0
10             0.70      0.0
5              0.68      0.0
1              0.57      0.0


Stat: 3pa
               corr  p-value
last                        
15_weighted    0.72      0.0
15_unweighted  0.72      0.0
10          

Index: []


Stat: blk_trend_10
Empty DataFrame
Columns: [corr, p-value]
Index: []


Stat: tov_trend_3
Empty DataFrame
Columns: [corr, p-value]
Index: []


Stat: tov_trend_5
Empty DataFrame
Columns: [corr, p-value]
Index: []


Stat: tov_trend_10
Empty DataFrame
Columns: [corr, p-value]
Index: []


Stat: pf_trend_3
Empty DataFrame
Columns: [corr, p-value]
Index: []


Stat: pf_trend_5
               corr  p-value
last                        
15_weighted   -0.02     0.03
15_unweighted -0.02     0.03
10            -0.02     0.03
5             -0.02     0.03
1             -0.02     0.03


Stat: pf_trend_10
               corr  p-value
last                        
15_weighted   -0.02     0.03
15_unweighted -0.02     0.03
10            -0.02     0.03
5             -0.02     0.03
1             -0.02     0.03


Stat: pts_trend_3
               corr  p-value
last                        
15_weighted    0.02     0.01
15_unweighted  0.02     0.01
10             0.02     0.01
5              0.02     